In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU,SimpleRNN
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping


In [2]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

REPLICAS:  1


In [3]:
train = pd.read_csv('../dataset/train_modified.csv')
valid = pd.read_csv('../dataset/val_modified.csv')
test = pd.read_csv('../dataset/test_modified.csv')

In [4]:
train

,Emotion,Comment
0,0,On days when I feel close to my partner and ot...
1,1,Every time I imagine that someone I love or I ...
2,2,When I had been obviously unjustly treated and...
3,3,When I think about the short time that we live...
4,0,After my girlfriend had taken her exam we went...
...,...,...
3000,1,"On Christmas eve,1984, I had just finished the..."
3001,2,"In September 1984, I was forced to live with s..."
3002,0,I received a letter from a distant friend.
3003,1,My parents were out and I was the eldest at ho...


In [5]:
#IMP DATA FOR CONFIG

AUTO = tf.data.experimental.AUTOTUNE


# Configuration
EPOCHS = 3
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
MAX_LEN = 168

In [6]:
xtrain, ytrain = train.Comment.values, train.Emotion.values
xvalid, yvalid = valid.Comment.values, valid.Emotion.values
xtest, ytest = test.Comment.values, test.Emotion.values
                                                  

train['Comment'].apply(lambda x:len(str(x).split())).max()

119

In [7]:
# using keras tokenizer here
token = text.Tokenizer(num_words=None)
max_len = 168

token.fit_on_texts(list(xtrain) + list(xvalid))
xtrain_seq = token.texts_to_sequences(xtrain)
xvalid_seq = token.texts_to_sequences(xvalid)
xtest_seq = token.texts_to_sequences(xtest)

#zero pad the sequences
xtrain_pad = sequence.pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = sequence.pad_sequences(xvalid_seq, maxlen=max_len)
xtest_pad = sequence.pad_sequences(xtest_seq, maxlen=max_len)

word_index = token.word_index

In [8]:
metrics = [tf.keras.metrics.SparseCategoricalAccuracy('accuracy', dtype=tf.float32)]
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [9]:
with strategy.scope():
    # A simpleRNN without any pretrained embeddings and one dense layer
    model = Sequential()
    model.add(Embedding(len(word_index) + 1,
                     300,
                     input_length=max_len))
    model.add(LSTM(128))
    model.add(Dense(7, activation='softmax'))
    model.compile(loss=loss, optimizer='adam', metrics=['accuracy'])
    
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 168, 300)          2860500   
_________________________________________________________________
lstm (LSTM)                  (None, 128)               219648    
_________________________________________________________________
dense (Dense)                (None, 7)                 903       
Total params: 3,081,051
Trainable params: 3,081,051
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.fit(xtrain_pad, ytrain,epochs=10,validation_data=(xvalid_pad, yvalid), batch_size=64*strategy.num_replicas_in_sync) #Multiplying by Strategy to run on TPU's

Epoch 1/10
47/47 [==============================] - 12s 250ms/step - loss: 1.8670 - accuracy: 0.2479 - val_loss: 1.8503 - val_accuracy: 0.2540
Epoch 2/10
47/47 [==============================] - 12s 259ms/step - loss: 1.7930 - accuracy: 0.3953 - val_loss: 1.8036 - val_accuracy: 0.3245
Epoch 3/10
47/47 [==============================] - 13s 271ms/step - loss: 1.5462 - accuracy: 0.6692 - val_loss: 1.7390 - val_accuracy: 0.4214
Epoch 4/10
47/47 [==============================] - 13s 276ms/step - loss: 1.3543 - accuracy: 0.8309 - val_loss: 1.7335 - val_accuracy: 0.4302
Epoch 5/10
47/47 [==============================] - 13s 279ms/step - loss: 1.2953 - accuracy: 0.8805 - val_loss: 1.7470 - val_accuracy: 0.4112
Epoch 6/10
47/47 [==============================] - 13s 279ms/step - loss: 1.2683 - accuracy: 0.9092 - val_loss: 1.7497 - val_accuracy: 0.4082
Epoch 7/10
47/47 [==============================] - 13s 274ms/step - loss: 1.2349 - accuracy: 0.9388 - val_loss: 1.7269 - val_accuracy: 0.4317

In [12]:
model.evaluate(xtest_pad, ytest)

21/21 [==============================] - 1s 49ms/step - loss: 1.7800 - accuracy: 0.3795


[1.7799577713012695, 0.3794712424278259]